# Similarité de compétences longues (missions)

Objectif: Construire une métrique de similarité entre compétences longues extraites de missions d'employés

Exercices:
1. Exploration et visualisation de la donnée
2. Traiter les missions pour extraire des phrases ou morceaux de phrases contenant des compétences
3. Trouver les 10 compétences longues les plus fréquentes sur tous les métiers
4. Visualiser les groupes de compétences longues similaires au sein d'un même profil
5. Visualiser les groupes de compétences longues similaires parmi les 40 comptétences longues les plus fréquentes de chaque métier

## Imports

In [1]:
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cdist
from unidecode import unidecode
from collections import defaultdict
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from typing import List, Dict, Any, Optional

from nltk import stem , Tree

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px

import re
import numpy as np
import pandas as pd
import spacy
import nltk
import os
import json

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/philippe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Loading data

In [2]:
df_profiles = pd.read_csv('employee_profiles.csv')
df_profiles.head(10)

,id_profile,jobtitle
0,918318,responsable de projet
1,835172,webmaster
2,814422,développeur
3,503545,responsable de projet
4,49282,responsable de projet
5,205700,responsable de projet
6,550243,responsable de projet
7,349518,responsable de projet
8,473990,responsable de projet
9,335697,responsable de projet


In [3]:
df_experiences = pd.read_csv('employee_experiences.csv')
df_experiences.head(10)

,id_profile,description,title,id_exp
0,918318,En tant que consultant SI interne au Groupe ED...,"Chef de projet, consultant en transformation d...",0
1,918318,En charge du déploiement d'un projet de signat...,Chef de projet,1
2,918318,Product Owner pour le compte de mc2i Groupe.Re...,Product Owner - Application mobile & tablette,2
3,918318,Consultant AMOA pour le premier organisme de c...,Gestion de projet,3
4,918318,Consultant AMOA pour un grand groupe audiovisu...,Consultant AMOA,4
5,918318,Consultant AMOA pour un grand groupe audiovisu...,Consultant AMOA,5
6,918318,Etude d'opportunité sur le Blended-Learning d...,Consultant SI,6
7,918318,NaN,Conseiller - Vendeur,7
8,918318,•\tRéalisation du 1er Job Dating en Malaisie•\...,Chargé de projet,8
9,918318,•\tOptimisation du site internet : www.skcb.cz...,Assistant marketing sportif,9
